In [1]:
import os
from crewai import Agent, Crew, Task, LLM, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv

/opt/anaconda3/envs/agents/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/agents/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/anaconda3/envs/agents/lib/python3.10/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. 

In [2]:
load_dotenv()
CEREBRAS_API_KEY = os.getenv("CEREBRAS_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [3]:
if not CEREBRAS_API_KEY:
    raise ValueError("Missing Cerebras API Key! Set CEREBRAS_API_KEY in environment variables.")

if not SERPER_API_KEY:
    raise ValueError("Missing Serper API Key! Set SERPER_API_KEY in environment variables.")

In [4]:
cerebras_llm = LLM(
    model="cerebras/llama-3.3-70b",
    temperature=0.7,
    max_tokens=8192,
    api_key=CEREBRAS_API_KEY,
    base_url="https://api.cerebras.ai/v1",
)

### Tools

In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Agents

In [19]:
search = Agent(
        role="E-Commerce Market Research Analyst",
        goal=f"Provide up-to-date market analysis of {product_name} from E-commerce Industry",
        backstory="An expert analyst with a keen eye for market trends",
        tools=[search_tool, scrape_tool],
        verbose=True,
        llm=cerebras_llm
    )

data_cleaner = Agent(
    role="Data Cleaning Specialist",
    goal=f"Ensure all price values for {product_name} are accurate, properly formatted, and free of inconsistencies.",
    backstory=(
        "An experienced data analyst with a strong background in data preprocessing, "
        "error detection, and price standardization. With expertise in handling messy datasets, "
        "you identify and clean incorrect, missing, or inconsistent price values, ensuring the data is reliable for further analysis."
    ),
    tools=[],
    verbose=True,
    llm=cerebras_llm
)

comparison = Agent(
    role="Price Comparison Expert",
    goal=f"Analyze and compare {product_name} prices to identify the lowest price available.",
    backstory=(
        "A meticulous price analyst with expertise in comparing product prices across different sources. "
        "You efficiently process pricing data, highlight discrepancies, and determine the best deal for consumers."
    ),
    tools=[],
    verbose=True,
    llm=cerebras_llm
)

reporting_agent = Agent(
    role="Market Insights Reporter",
    goal=f"Generate a comprehensive report summarizing price trends, differences, and the best available deals for {product_name}.",
    backstory=(
        "A skilled data journalist with experience in analyzing pricing trends and market fluctuations. "
        "You transform raw pricing data into insightful reports, providing actionable insights on cost-effective options."
    ),
    tools=[],
    verbose=True,
    llm=cerebras_llm
)

### Tasks 

In [20]:
search_task = Task(
    description=f"Collect current pricing data for {product_name} from at least 3 major e-commerce platforms. Include product name, model, specifications, price, and any ongoing promotions or discounts.",
    expected_output=f"A structured dataset containing {product_name} information and pricing from multiple sources, with complete pricing details.",
    agent=search
)

cleaning_task = Task(
    description=f"Process the raw pricing data for {product_name} to standardize formats, handle currency conversions, remove outliers, and identify any inconsistencies or errors in the collected price information.",
    expected_output=f"A cleaned dataset with uniformly formatted prices for {product_name}, standardized currencies, and annotations for any identified anomalies or special pricing conditions.",
    agent=data_cleaner
)

comparison_task = Task(
    description=f"Analyze the cleaned pricing data to identify the lowest available price for {product_name}, calculate price differences between retailers, and determine price-to-value ratios based on product specifications.",
    expected_output=f"A comparative analysis showing price rankings for {product_name}, percentage differences between retailers, and identification of the best value options across different price points.",
    agent=comparison
)

reporting_task = Task(
    description=f"Create a comprehensive market insights report based on the {product_name} pricing analysis, highlighting best deals, pricing trends, and actionable recommendations for price-conscious consumers.",
    expected_output=f"A detailed report for {product_name} with executive summary, visualizations of price comparisons, identification of pricing patterns, and specific recommendations for optimal purchasing decisions.",
    agent=reporting_agent
)

In [21]:
product_price_crew = Crew(
    agents=[search, data_cleaner, comparison, reporting_agent],
    tasks=[search_task, cleaning_task, comparison_task, reporting_task], 
    verbose=True,
    process=Process.sequential,
)

2025-02-25 03:19:07,616 - 8480639040 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
product_name = "Sony WH-1000XM5"
country = "United States"
# format = {'product': product_name, 'country': country}
format = {'product': product_name}

In [23]:
product_name = "Lenovo Earbuds LP40"
country = "United States"
format = {'product': product_name, 'country': country}

In [26]:
# Execute Crew
event_analysis = product_price_crew.kickoff(inputs=format)
# Print the final report
print(event_analysis)

# Agent: E-Commerce Market Research Analyst
## Task: Collect current pricing data for Lenovo Earbuds LP40 from at least 3 major e-commerce platforms. Include product name, model, specifications, price, and any ongoing promotions or discounts.


# Agent: E-Commerce Market Research Analyst
## Thought: Thought: To collect current pricing data for Lenovo Earbuds LP40 from at least 3 major e-commerce platforms, I should start by searching for the product on popular e-commerce websites such as Amazon, Best Buy, and Walmart. This will provide me with the product name, model, specifications, price, and any ongoing promotions or discounts.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Lenovo Earbuds LP40 price comparison\"}"
## Tool Output: 
{'searchParameters': {'q': 'Lenovo Earbuds LP40 price comparison', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Lenovo Lp40 - Amazon.com', 'link': 'https://www.amazon.com/lenovo-lp40/s?k=len